<a href="https://colab.research.google.com/github/AnimeshKrai/Repository-Demo/blob/main/IITJodhpurHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 15.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

df = load_dataset("zpn/clintox")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

(…)-00000-of-00001-fa296d8e8d8c9c07.parquet:   0%|          | 0.00/92.0k [00:00<?, ?B/s]

(…)-00000-of-00001-ca5407451a5f9454.parquet:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

(…)-00000-of-00001-6e352a5dc26e9a8a.parquet:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1181 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/148 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148 [00:00<?, ? examples/s]

#### **Loading the dependencies**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import binary_crossentropy
from sklearn.metrics import classification_report

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### **Loading and preparing the data**


In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 1181
    })
    validation: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
    test: Dataset({
        features: ['smiles', 'selfies', 'target'],
        num_rows: 148
    })
})

In [ ]:
train = df['train']
test = df['test']
valid = df['validation']

train = pd.DataFrame(train)
test = pd.DataFrame(test)
valid = pd.DataFrame(valid)

In [ ]:
train

,smiles,selfies,target
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,[C][C][Branch1][C][C][C][C@H1][Branch2][Ring1]...,1
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,[O][=C][Branch1][=Branch2][N][C][C][Branch1][C...,0
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,[Cl][C][=C][C][Branch1][C][Cl][=C][Branch1][=B...,0
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,[N][#C][C][=C][C][Branch1][O][N][C][=Branch1][...,0
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,[N][S][=Branch1][C][=O][=Branch1][C][=O][C][=C...,0
...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,[C][C][Branch1][O][C][NH2+1][C][C][C][C][C][C]...,0
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,[C][C][Branch1][=Branch1][C][=Branch1][C][=O][...,0
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,[C][C][Branch1][=C][C][=C][C][=C][C][=C][C][=C...,0
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,[C][C][Branch1][C][O][C][Branch1][Ring1][C][O]...,0


In [ ]:
train[train["target"]==1]

,smiles,selfies,target
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,[C][C][Branch1][C][C][C][C@H1][Branch2][Ring1]...,1
32,O=C(CCCCCCC(=O)Nc1ccccc1)NO,[O][=C][Branch2][Ring1][Ring2][C][C][C][C][C][...,1
88,CC(=O)Oc1ccccc1C(=O)O,[C][C][=Branch1][C][=O][O][C][=C][C][=C][C][=C...,1
129,CCN(CC)CC(=O)Nc1c(C)cccc1C,[C][C][N][Branch1][Ring1][C][C][C][C][=Branch1...,1
152,[Se],[Se],1
...,...,...,...
1122,Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(C...,[C][C][=N][C][Branch2][Ring1][=N][N][C][=N][C]...,1
1123,Cc1cccc(C)c1N(CC(=O)Nc1ccc(-c2ncon2)cc1)C(=O)C...,[C][C][=C][C][=C][C][Branch1][C][C][=C][Ring1]...,1
1126,COc1c(C)c2c(c(O)c1C/C=C(\C)CCC(=O)OCCN1CCOCC1)...,[C][O][C][=C][Branch1][C][C][C][=C][Branch2][R...,1
1133,CC1(O)CC(c2nc(-c3ccc4ccc(-c5ccccc5)nc4c3)c3c(N...,[C][C][Branch1][C][O][C][C][Branch2][Ring2][=N...,1


In [ ]:
train[train["target"]==0]

,smiles,selfies,target
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,[O][=C][Branch1][=Branch2][N][C][C][Branch1][C...,0
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,[Cl][C][=C][C][Branch1][C][Cl][=C][Branch1][=B...,0
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,[N][#C][C][=C][C][Branch1][O][N][C][=Branch1][...,0
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,[N][S][=Branch1][C][=O][=Branch1][C][=O][C][=C...,0
5,NC(N)=[NH+]C(=O)Cc1c(Cl)cccc1Cl,[N][C][Branch1][C][N][=NH1+1][C][=Branch1][C][...,0
...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,[C][C][Branch1][O][C][NH2+1][C][C][C][C][C][C]...,0
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,[C][C][Branch1][=Branch1][C][=Branch1][C][=O][...,0
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,[C][C][Branch1][=C][C][=C][C][=C][C][=C][C][=C...,0
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,[C][C][Branch1][C][O][C][Branch1][Ring1][C][O]...,0


In [ ]:
test

,smiles,selfies,target
0,C[NH+]1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4C...,[C][NH1+1][C][C@H1][Branch2][Ring2][#C][C][=Br...,0
1,C[C@]1(Cn2ccnn2)[C@H](C(=O)[O-])N2C(=O)C[C@H]2...,[C][C@][Branch1][=Branch2][C][N][C][=C][N][=N]...,0
2,C[NH+]1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1,[C][NH1+1][C][C][C][C@@H1][Ring1][Branch1][C][...,0
3,Nc1nc(NC2CC2)c2ncn([C@H]3C=C[C@@H](CO)C3)c2n1,[N][C][=N][C][Branch1][#Branch1][N][C][C][C][R...,0
4,OC[C@H]1O[C@@H](n2cnc3c2NC=[NH+]C[C@H]3O)C[C@@...,[O][C][C@H1][O][C@@H1][Branch1][S][N][C][=N][C...,0
...,...,...,...
143,O=C1O[C@H]([C@@H](O)CO)C([O-])=C1O,[O][=C][O][C@H1][Branch1][#Branch1][C@@H1][Bra...,0
144,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)N[C@@H]...,[C][#C][C][C][Branch2][Ring1][=Branch1][C][C][...,0
145,C#CC[NH2+][C@@H]1CCc2ccccc21,[C][#C][C][NH2+1][C@@H1][C][C][C][=C][C][=C][C...,0
146,[H]/[NH+]=C(\N)c1ccc(OCCCCCOc2ccc(/C(N)=[NH+]/...,[H][/NH1+1][=C][Branch1][C][\N][C][=C][C][=C][...,0


In [ ]:
valid

,smiles,selfies,target
0,CC(C)OC(=O)CCC/C=C\C[C@@H]1[C@@H](/C=C/[C@@H](...,[C][C][Branch1][C][C][O][C][=Branch1][C][=O][C...,0
1,CC(C)Nc1cccnc1N1CCN(C(=O)c2cc3cc(NS(C)(=O)=O)c...,[C][C][Branch1][C][C][N][C][=C][C][=C][N][=C][...,0
2,CC(C)n1c(/C=C/[C@H](O)C[C@H](O)CC(=O)[O-])c(-c...,[C][C][Branch1][C][C][N][C][Branch2][Ring1][C]...,0
3,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)[NH+]1CCCC1,[C][C][Branch1][C][C][C][O][C][C][Branch2][Rin...,0
4,CSCC[C@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)NC(=O)...,[C][S][C][C][C@H1][Branch2][Ring2][#Branch2][N...,0
...,...,...,...
143,C[C@H](OC(=O)c1ccccc1)C1=CCC23OCC[NH+](C)CC12C...,[C][C@H1][Branch1][=C][O][C][=Branch1][C][=O][...,0
144,C[C@@H](c1ncncc1F)[C@](O)(Cn1cncn1)c1ccc(F)cc1F,[C][C@@H1][Branch1][#Branch2][C][=N][C][=N][C]...,0
145,CC(C)C[C@@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@@H]...,[C][C][Branch1][C][C][C][C@@H1][Branch2][Ring1...,0
146,C[C@H](O)[C@H](O)[C@H]1CNc2[nH]c(N)nc(=O)c2N1,[C][C@H1][Branch1][C][O][C@H1][Branch1][C][O][...,0


In [ ]:
train = train.drop(['selfies'], axis = 1) #We don't require this column

In [ ]:
#generating and extracting features from the
train['mol'] = train['smiles'].apply(Chem.MolFromSmiles)
test['mol'] = test['smiles'].apply(Chem.MolFromSmiles)
valid['mol'] = valid['smiles'].apply(Chem.MolFromSmiles)
# train['mol_wt'] = train['mol'].apply(lambda x: Descriptors.MolWt(x) if x is not None else None)
# train['logp'] = train['mol'].apply(lambda x: Descriptors.MolLogP(x) if x is not None else None)
# train['tpsa'] = train['mol'].apply(lambda x: Descriptors.TPSA(x) if x is not None else None)
# train['hbd'] = train['mol'].apply(lambda x: Descriptors.NumHDonors(x) if x is not None else None)
# train['hba'] = train['mol'].apply(lambda x: Descriptors.NumHAcceptors(x) if x is not None else None)
# train['rot_bonds'] = train['mol'].apply(lambda x: Descriptors.NumRotatableBonds(x) if x is not None else None)
# train['ring_count'] = train['mol'].apply(lambda x: Descriptors.RingCount(x) if x is not None else None)
# train['aromaticity'] = train['mol'].apply(lambda x: Descriptors.NumAromaticRings(x) if x is not None else None)

In [ ]:
train

,smiles,target,mol
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,1,<rdkit.Chem.rdchem.Mol object at 0x7cd50454dd90>
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d700>
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d7e0>
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d230>
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454cba0>
...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581b60>
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581bd0>
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581c40>
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581cb0>


## **Exploring the Data**


In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
descr = rdkit.Chem.Descriptors.CalcMolDescriptors("CCC", missingVal=None, silent=True)

In [ ]:
descr

{'MaxAbsEStateIndex': None,
 'MaxEStateIndex': None,
 'MinAbsEStateIndex': None,
 'MinEStateIndex': None,
 'qed': None,
 'SPS': None,
 'MolWt': None,
 'HeavyAtomMolWt': None,
 'ExactMolWt': None,
 'NumValenceElectrons': None,
 'NumRadicalElectrons': None,
 'MaxPartialCharge': None,
 'MinPartialCharge': None,
 'MaxAbsPartialCharge': None,
 'MinAbsPartialCharge': None,
 'FpDensityMorgan1': None,
 'FpDensityMorgan2': None,
 'FpDensityMorgan3': None,
 'BCUT2D_MWHI': nan,
 'BCUT2D_MWLOW': nan,
 'BCUT2D_CHGHI': nan,
 'BCUT2D_CHGLO': nan,
 'BCUT2D_LOGPHI': nan,
 'BCUT2D_LOGPLOW': nan,
 'BCUT2D_MRHI': nan,
 'BCUT2D_MRLOW': nan,
 'AvgIpc': None,
 'BalabanJ': None,
 'BertzCT': None,
 'Chi0': None,
 'Chi0n': None,
 'Chi0v': None,
 'Chi1': None,
 'Chi1n': None,
 'Chi1v': None,
 'Chi2n': None,
 'Chi2v': None,
 'Chi3n': None,
 'Chi3v': None,
 'Chi4n': None,
 'Chi4v': None,
 'HallKierAlpha': None,
 'Ipc': None,
 'Kappa1': None,
 'Kappa2': None,
 'Kappa3': None,
 'LabuteASA': None,
 'PEOE_VSA1': None,

In [ ]:
listofdescr = list(descr.keys())

In [ ]:
listofdescr

['MaxAbsEStateIndex',
 'MaxEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'qed',
 'SPS',
 'MolWt',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'NumRadicalElectrons',
 'MaxPartialCharge',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'MinAbsPartialCharge',
 'FpDensityMorgan1',
 'FpDensityMorgan2',
 'FpDensityMorgan3',
 'BCUT2D_MWHI',
 'BCUT2D_MWLOW',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPHI',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRHI',
 'BCUT2D_MRLOW',
 'AvgIpc',
 'BalabanJ',
 'BertzCT',
 'Chi0',
 'Chi0n',
 'Chi0v',
 'Chi1',
 'Chi1n',
 'Chi1v',
 'Chi2n',
 'Chi2v',
 'Chi3n',
 'Chi3v',
 'Chi4n',
 'Chi4v',
 'HallKierAlpha',
 'Ipc',
 'Kappa1',
 'Kappa2',
 'Kappa3',
 'LabuteASA',
 'PEOE_VSA1',
 'PEOE_VSA10',
 'PEOE_VSA11',
 'PEOE_VSA12',
 'PEOE_VSA13',
 'PEOE_VSA14',
 'PEOE_VSA2',
 'PEOE_VSA3',
 'PEOE_VSA4',
 'PEOE_VSA5',
 'PEOE_VSA6',
 'PEOE_VSA7',
 'PEOE_VSA8',
 'PEOE_VSA9',
 'SMR_VSA1',
 'SMR_VSA10',
 'SMR_VSA2',
 'SMR_VSA3',
 'SMR_VSA4',
 'SMR_VSA5',
 'SMR_VSA6',


In [ ]:
for l in listofdescr:
  train[str(l)]=None

In [ ]:
train

,smiles,target,mol,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,1,<rdkit.Chem.rdchem.Mol object at 0x7cd50454dd90>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO...,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d700>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Clc1cc(Cl)c(OCC#CI)cc1Cl,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d7e0>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,N#Cc1cc(NC(=O)C(=O)[O-])c(Cl)c(NC(=O)C(=O)[O-])c1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454d230>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd50454cba0>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,CC(C[NH2+]C1CCCCC1)OC(=O)c1ccccc1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581b60>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1177,CC(C(=O)[O-])c1ccc(C(=O)c2cccs2)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581bd0>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1178,CC(c1cc2ccccc2s1)N(O)C(N)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581c40>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1179,CC(O)C(CO)NC(=O)C1CSSCC(NC(=O)C([NH3+])Cc2cccc...,0,<rdkit.Chem.rdchem.Mol object at 0x7cd504581cb0>,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
len(listofdescr)

210

In [ ]:

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors



j = 0
for mol in train["mol"]:
    descr = Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
    descr_values = list(descr.values())

    # Replace NaN values with 0
    descr_values = [0 if pd.isna(d) else float(d) for d in descr_values]

    # Update DataFrame using .iat accessor
    for i, val in enumerate(descr_values):
        train.iat[j, i + 3] = val

    j += 1


[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerator
[13:24:04] DEPRECATION WARNING: please use MorganGenerat

In [ ]:
for d in listofdescr:
  train[str(d)] = train[str(d)].fillna(0).astype(float)

# descr

In [ ]:
train

In [ ]:
train['target'].dtype

In [ ]:
train['MolWt'].dtype


In [ ]:
correlationlist=[]
for d in listofdescr:
  corr=train[str(d)].corr(train["target"])
  correlationlist.append(corr)

In [ ]:
correlationlist

In [ ]:
j=0
dictDescr={}
for li in listofdescr:
  dictDescr[str(li)]= str(correlationlist[j])
  j=j+1

In [ ]:
dictDescr

In [ ]:
relevantDiscr=[]
for k,v in dictDescr.items():
  if (float(v) > 0.1):
    relevantDiscr.append(k)

In [ ]:
relevantDiscr

In [ ]:
for d in listofdescr:
  if d not in relevantDiscr:
    train=train.drop([str(d)], axis=1)

In [ ]:
train

## **Applying the same changes to the train, test and validate datasets**

In [ ]:
train

In [ ]:
test

In [ ]:
test=test.drop(["selfies"], axis =1)

In [ ]:
listofdescr = list(descr.keys())
for l in listofdescr:
  test[str(l)]=None

j = 0
for mol in test["mol"]:
    descr = Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
    descr_values = list(descr.values())

    # Replace NaN values with 0
    descr_values = [0 if pd.isna(d) else float(d) for d in descr_values]

    # Update DataFrame using .iat accessor
    for i, val in enumerate(descr_values):
        test.iat[j, i + 3] = val

    j += 1

for d in listofdescr:
  test[str(d)] = test[str(d)].fillna(0).astype(float)

In [ ]:
test

In [ ]:
for d in listofdescr:
  if d not in relevantDiscr:
    test=test.drop([str(d)], axis=1)

In [ ]:
test

In [ ]:
valid=valid.drop(["selfies"],axis=1)

listofdescr = list(descr.keys())
for l in listofdescr:
  valid[str(l)]=None

j = 0
for mol in valid["mol"]:
    descr = Chem.Descriptors.CalcMolDescriptors(mol, missingVal=None, silent=True)
    descr_values = list(descr.values())

    # Replace NaN values with 0
    descr_values = [0 if pd.isna(d) else float(d) for d in descr_values]

    # Update DataFrame using .iat accessor
    for i, val in enumerate(descr_values):
        valid.iat[j, i + 3] = val

    j += 1

for d in listofdescr:
  valid[str(d)] = valid[str(d)].fillna(0).astype(float)

In [ ]:
for d in listofdescr:
  if d not in relevantDiscr:
    valid=valid.drop([str(d)], axis=1)

In [ ]:
# adding same to validate set

In [ ]:
toxicophores = {
    'Aldehydes_Ketones': 'C(=O)',
    'Organophosphates': 'P(=O)(OCC)OCC',
    'Isocyanates': 'C(=O)N=C=O',
    'Heavy_Metals_Mercury': '[Hg]',
    'Aromatic_Hydrocarbons': 'c1ccccc1',
    'Nitriles': 'CC#N',
    'PAHs': 'c1ccc2cc3ccccc3cc2c1',
    'Ethers': 'CCOCC',
    'Azides': 'CC(=O)N=[N+]=[N-]',
    'Halogens_Chlorine': 'Cl'
}

# Create binary columns for each toxicophore
for toxicophore_name, smart_pattern in toxicophores.items():
    train[toxicophore_name] = train['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)
    test[toxicophore_name] = test['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)
    valid[toxicophore_name] = valid['mol'].apply(lambda x: 1 if x is not None and x.HasSubstructMatch(Chem.MolFromSmarts(smart_pattern)) else 0)


In [ ]:
test=test.drop(["smiles","mol"],axis=1)
test

In [ ]:
valid=valid.drop(["smiles","mol"],axis=1)
valid

In [ ]:
train =  train.drop(['smiles', 'mol'], axis = 1)

In [ ]:
x_train = train.drop(['target'], axis = 1 )
y_train = train['target']

x_valid = valid.drop(['target'], axis = 1)
y_valid = valid['target']

x_test = test.drop(['target'], axis =1)
y_test = test['target']

In [ ]:
y_train.value_counts()

## **start of GRAPHS**

In [ ]:
train.columns

In [ ]:
len(train.columns)

In [ ]:
xaxis=[]
yaxis="target"
for i in train.columns:
  if (i != "target"):
    xaxis.append(i)

In [ ]:
sns.pairplot(train, hue='target')
plt.title('Pair Plot: Features by Toxicity')
plt.show()


### **Correlation Heatmap**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame with the specified columns
corr_matrix = train[train.columns].corr()
plt.figure(figsize=(36, 30))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
for feature in xaxis:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='target', y=feature, data=train)
    plt.xlabel('toxicity')
    plt.ylabel(feature)
    plt.title(f'Scatter Plot: {feature} vs Toxicity ')
    plt.show()

In [ ]:
for feature in xaxis:
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='target', y=feature, data=train)
    plt.xlabel('Toxicity')
    plt.ylabel(feature)
    plt.title(f'Box Plot: {feature} by Toxicity')
    plt.show()

In [ ]:
for feature in xaxis:
    plt.figure(figsize=(10, 6))
    sns.histplot(train, x=feature, bins=30, hue='target', multiple='stack')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.title(f'Histogram: {feature} Distribution by Toxicity')
    plt.show()

In [ ]:
num_features = len(xaxis)
num_cols = 3

num_rows = (num_features + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))

for i, feature in enumerate(xaxis):
    row = i // num_cols
    col = i % num_cols

    sns.histplot(train, x=feature, bins=30, hue='target', multiple='stack', ax=axes[row, col])
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].set_title(f'Histogram: {feature} Distribution by Toxicity')

plt.tight_layout()
plt.show()

## **end of GRAPHS**

# **Model Developement**


In [ ]:
#training the model by undersampling to reduce the class imbalance

sampl = RandomUnderSampler(sampling_strategy='auto', random_state=42)
x_resampled, y_resampled = sampl.fit_resample(x_train, y_train)

clf1 = LogisticRegression()
clf2 = SVC()
clf3 = KNeighborsClassifier()
clf4 = DecisionTreeClassifier()
clf5 = RandomForestClassifier()
clf6 = AdaBoostClassifier()
clf7 = GradientBoostingClassifier()
clf8 = XGBClassifier()

classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8]

for var in classifiers:
  var.fit(x_resampled, y_resampled)
  y_pred = var.predict(x_valid)

  print("\n")
  print(f"The Classification report of {var} is: \n {classification_report(y_valid, y_pred)} ")
  print(f"The AUC-ROC score: {roc_auc_score(y_valid, y_pred)}")
  print(f"The AUC-PRC Score: {average_precision_score(y_valid, y_pred)}")
  print("\n")

In [ ]:
#training the model by oversampling to reduce the class imbalance
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score


sampl = ADASYN(sampling_strategy='auto', random_state=42)
x_resampled1, y_resampled1 = sampl.fit_resample(x_train, y_train)

clf1 = LogisticRegression()
clf2 = SVC()
clf3 = KNeighborsClassifier()
clf4 = DecisionTreeClassifier()
clf5 = RandomForestClassifier()
clf6 = AdaBoostClassifier()
clf7 = GradientBoostingClassifier()
clf8 = XGBClassifier()

classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8]

for var in classifiers:
  var.fit(x_resampled1, y_resampled1)
  y_pred = var.predict(x_valid)

  print("\n")
  print(f"The Classification report of {var} is: \n {classification_report(y_valid, y_pred)} ")
  print(f"The AUC-ROC score: {roc_auc_score(y_valid, y_pred)}")
  print(f"The AUC-PRC Score: {average_precision_score(y_valid, y_pred)}")
  print("\n")

In [ ]:
x_train.shape

## **Neural Networks**

After applying every single supervised model, we decided to apply neural networks on it.


In [ ]:
ann = Sequential()
ann.add(Dense(32, kernel_initializer = 'he_normal', input_dim = 32, activation = 'relu'))
ann.add(Dropout(0.2))
ann.add(Dense(16, kernel_initializer = 'he_normal',  activation = 'relu'))
ann.add(Dropout(0.2))
ann.add(Dense(1, kernel_initializer = 'he_normal', activation = 'sigmoid'))

In [ ]:
ann.summary()

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann.fit(x_train, y_train, epochs = 20, batch_size = 8, validation_data = (x_valid, y_valid))

As we can cleary see, the data available for training is very less to be fed into a neural network, thus, we shall only stick to ML models.

## **Fine tuning the model**

In [ ]:
#We are choosing XGboost on undersampled data as it is performing the best

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

grid_search = GridSearchCV(
    clf8,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
)

grid_search.fit(x_resampled , y_resampled)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
best_params

In [ ]:
#Our final model
best_model.fit(x_resampled, y_resampled)
y_pred = best_model.predict(x_valid)

print("\n")
print(f"The Classification report of fine tuned model is :\n {classification_report(y_valid, y_pred)}")
print("\n")
print(f"The AUC-ROC score of the fine tuned model is: {roc_auc_score(y_valid, y_pred)}")